In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score, precision_recall_curve,auc
import seaborn as sns
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


## 1. Data preprocessing

- The merged dataset containers data leakage features which are features that wouldnt be available in real-world prediction such as `max_delay`, `avg_delay`, `num_bad_months`, `months_total`. The target variable `label`, `id` and `amt_income_total` (to use the log transformed income) are also dropped. The target variable `label` is converted to integer for classification
- Applied one-hot encoding to categorical variables 
- Applied train-test split and SMOTE

In [ ]:
df = pd.read_csv("../data/clean_merged.csv")


In [ ]:
y = df['label'].astype('int')
leaky_features = ['max_delay', 'avg_delay', 'num_bad_months', 'months_total']
X = df.drop(columns=['label', 'id', 'amt_income_total'] + leaky_features)



In [ ]:
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
if len(categorical_cols) > 0:
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True) #one-hot encoding
else:
    X_encoded = X

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.head()

In [ ]:
X_train.info()

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# apply SMOTE 
smote = SMOTE(random_state=42)
X_balance, y_balance = smote.fit_resample(X_train, y_train)
print(f"Class distribution of y after SMOTE: {Counter(y_balance)}")
print(f"Class distribution of y without SMOTE: {Counter(y_test)}")

## 2. Random Forest Model (Baseline)

### 2.1 Baseline model

Random Forest is an ensemble learning method based on building multiple decision trees during training and combining their predictions by majority vote for classification. 

This makes the model:
- Generalise better and avoid overfitting
- Handle non-linear relationships
- Automatically estimate feature importance

The first model is a baseline model that is simple and unoptimised version that uses the original imbalanced data with default Random Forest hyperparameters to provide a reference point.

In [ ]:
# Train Random forest classifier
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 42)
rf_model.fit(X_train, y_train)

# Make predictions on test set
y_pred = rf_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))  # y_test loaded as true test targets
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

### 2.2 Baseline Model (using only the top 10 most important features)

Rationale for reducing to top 10 most important features:
- Interpretability 
- Eliminates noisy or irrelevant features that might affect accuracy of model
- Reduce likelihood of model learning random patterns 
- Fewer inputs = Increase training efficiency

In [ ]:
# Get feature importances and feature names
importances = rf_model.feature_importances_
feature_names = X_train.columns

# Make a DataFrame for easy sorting/viewing
feat_imp_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

feat_imp_df

In [ ]:
# Plot top N features (e.g. top 15)
N = 15
plt.figure(figsize=(10, 6))
plt.barh(feat_imp_df['feature'][:N][::-1], feat_imp_df['importance'][:N][::-1])
plt.xlabel('Importance')
plt.title('Top Feature Importances from Random Forest')
plt.tight_layout()
plt.show()

In [ ]:
print(feat_imp_df.head(10))  # Show top 10 important features

In [ ]:
# reduce train and test to selected features

N = 10
top_features = feat_imp_df['feature'].iloc[:N].tolist()
top_features

X_train_reduced = X_train[top_features]
X_test_reduced = X_test[top_features]

In [ ]:
rf_reduced = RandomForestClassifier(n_estimators=100, random_state=42)
rf_reduced.fit(X_train_reduced, y_train)

y_pred_reduced = rf_reduced.predict(X_test_reduced)

print("Accuracy with reduced features:", accuracy_score(y_test, y_pred_reduced))

print("\nClassification Report:\n", classification_report(y_test, y_pred_reduced))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_reduced))

After feature reduction:
- Accuracy improved slightly: 0.9809380142622052 to 0.9810751508502469 
- Recall for target label 0 (no approval) is still 1.00
- Recall for target label 1 (approval) improved slightly: 0.15 to 0.16


In [ ]:
y_proba_reduced = rf_reduced.predict_proba(X_test_reduced)[:, 1]  # Probability of class '1'
fpr, tpr, thresholds = roc_curve(y_test, y_proba_reduced)  # y_test = true labels
# Calculate AUC
auc_score = roc_auc_score(y_test, y_proba_reduced)
print(f"AUC: {auc_score:.3f}")

In [ ]:
# Plot the ROC Curve
plt.figure(figsize = (6, 5))
plt.plot(fpr, tpr, label = f'ROC Curve (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label = 'Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Random Forest')
plt.legend()
plt.tight_layout()
plt.show()

### 2.3 GridSearchCV to find the best combination of hyperparameters for Random Forest Model 

Rationale for GridSearchCV:
- We want to improve model performance (measured by ROC-AUC) by testing different parameter settings (n_estimators, max_depth, min_samples_split) systematically.

Key Random Forest hyperparameters:
- `n_estimators`: Number of trees in the Random Forest. Having more trees can improve model performance but increase training time. 
- `max_depth`: Maximum depth of each tree which controls the model complexity
- `min_samples_split`: Minimum samples required to split a node. Larger `min_samples_split` prevents overfitting as it requires more samples to create a split.

#### How the GridSearchCV works?

GridSearchCV tries every combination in the grid, using cross-validation (CV) to evaluate each combination. We chose cv = 3 (3-fold CV splits), splits the data into 3 parts. The model is trained 3 rounds, each round, the model is trained on a different 2/3 of the data. This CV ensures the model generalises well.

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state = 42),
    param_grid,
    cv = 3,
    scoring = 'roc_auc',
    n_jobs = 1,       # <<< Only use 1 job (no parallel, less stress on env)
    verbose = 2
)
grid_rf.fit(X_train_reduced, y_train)

In [ ]:
print("Best Random Forest params:", grid_rf.best_params_)
print("Best cross-validated AUC:", grid_rf.best_score_)

The performance of the optimal hyperparameter combination identified by GridSearchCV was evaluated using the ROC-AUC metric.

ROC-AUC measures how well the Random Forest model separates the 2 classes (defaulters vs. non-defaulters)

Rationale for using ROC-AUC:
- We have identified the dataset to be imbalanced, so using accuracy metric can be misleading. A model that always predicts the majority class could still appear "accurate", even though it performs poorly on the minority class. Thus, the model might fail to identify important minority cases, like defaulters, which could cause harm to the bank.
- In contrast, ROC-AUC looks at how well the model separates defaulters vs. non-defaulters, so it is much more reliable compared to accuracy metric.

In [ ]:
# Evaluate the Best Model on Test Set
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Predict probabilities (for AUC) and classes
y_proba = grid_rf.best_estimator_.predict_proba(X_test_reduced)[:, 1]
y_pred = grid_rf.best_estimator_.predict(X_test_reduced)

# Evaluate
print("Test set ROC AUC:", roc_auc_score(y_test, y_proba))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Based on the Classification Report, even though the ROC-AUC shows a high value of 0.80, the recall value is low, and false negative rate is very high (0.88). In the financial institution context, type II error which is the false negative rate might be a greater consequence. The model would approve a credit card for someone who is actually a bad customer (likely to default) the bank loses money if the applicant with a poor credit history gets approved, and later fails to pay. Hence, to improve on this, section 3 contains different enhancements made to the baseline model to improve the performance metrics.

## 3. Random Forest Model Enhancements / Variants

### 3.1 Random Forest Model with SMOTE and Fixed Hyperparameters

In [ ]:

# Initialise Random Forest 
rf_model_smote = RandomForestClassifier(
    n_estimators = 100,        # no. of trees
    max_depth = 10,           
    min_samples_split = 20,   
    min_samples_leaf = 10,
    max_features = 'sqrt',     # random feature selection, diff trees see different subset of features 
    bootstrap = True,          # bagging, ensemble learning
    random_state = 42,
    n_jobs = -1               # for parallel processing
)

rf_model_smote.fit(X_balance, y_balance)

#  predictions
y_pred_smote = rf_model_smote.predict(X_test)
y_pred_proba_smote = rf_model_smote.predict_proba(X_test)[:, 1]

# performance metrics 
precision, recall, threshold = precision_recall_curve(y_test, y_pred_proba_smote)
pr_auc = auc(recall, precision)


print(f"Accuracy: {accuracy_score(y_test, y_pred_smote):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_smote):.4f}")
print(f"PR AUC: {pr_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_smote))

# confusion Matrix
cm = confusion_matrix(y_test, y_pred_smote)
print("Confusion Matrix:")
print(cm)

print(f"\nRates:")
total_0 = cm[0,0] + cm[0,1]  
total_1 = cm[1,0] + cm[1,1] 


print(f"\nSpecificity (True Negative Rate): {cm[0,0]/total_0:.4f}")
print(f"False Positive Rate: {cm[0,1]/total_0:.4f}")
print(f"Sensitivity (Recall/True Positive Rate): {cm[1,1]/total_1:.4f}")
print(f"False Negative Rate: {cm[1,0]/total_1:.4f}")



In [ ]:
# Plot ROC curve 
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_smote)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_score(y_test, y_pred_proba_smote):.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Random Forest')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

Issues with random forest model using training set balanced using smote:

1. AUC of 0.6 is only slightly better than random guesssing, contradicts the 0.90 accuracy rate. The accuracy rate might be misleading because majority class dominates (`label` = 0 for non default). The model might not be discriminating well between the 2 classes. 

2. Although recall increased from the baseline model (0.20 from 0.11), the trade off is lower precision and f1 score as these are nearly 0 for `label` = 1. Since the minority class is extremely small (about 0.02%), SMOTE might produce unrealisitc synthetic minority points, and the model might be overfitting to the SMOTE-generated data by learning the artificial patterns created by SMOTE but not generalizing well to the test set. Some features are dominated by 0s or categorical dummies.

3. Potential changes made to this model: 
 - to lower the threshold to 0.3 from 0.5 to capture more defaulters (true positives) (refer to section 3.2)
 - to change to class_weight="balanced" to replace SMOTE (refer to section 3.4)
 - change the model with features that are engineered (refer to section 3.4)

### 3.2 Random Forest Model (adjusted threshold)

In [ ]:
rf_model_thres = RandomForestClassifier(
    n_estimators=100,        # no. of trees
    max_depth=10,           
    min_samples_split=20,   
    min_samples_leaf=10,
    max_features='sqrt',     # random feature selection, diff trees see different subset of features 
    bootstrap=True,          # bagging, ensemble learning
    random_state=42,
    n_jobs=-1               # for parallel processing
)

rf_model_thres.fit(X_balance, y_balance)

y_pred_thres = rf_model_thres.predict(X_test)
y_pred_proba_thres = rf_model_thres.predict_proba(X_test)[:, 1]
threshold = 0.3
y_pred_adj = (y_pred_proba_thres >= threshold).astype(int)


# performance metrics 
precision, recall, threshold = precision_recall_curve(y_test, y_pred_proba_thres)
pr_auc = auc(recall, precision)


print(f"Accuracy: {accuracy_score(y_test, y_pred_adj):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_thres):.4f}")
print(f"PR AUC: {pr_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_adj))

# confusion Matrix
cm = confusion_matrix(y_test, y_pred_adj)
print("Confusion Matrix:")
print(cm)

print(f"\nRates:")
total_0 = cm[0,0] + cm[0,1]  
total_1 = cm[1,0] + cm[1,1] 

print(f"\nSpecificity (True Negative Rate): {cm[0,0]/total_0:.4f}")
print(f"False Positive Rate: {cm[0,1]/total_0:.4f}")
print(f"Sensitivity (Recall/True Positive Rate): {cm[1,1]/total_1:.4f}")
print(f"False Negative Rate: {cm[1,0]/total_1:.4f}")


Recall for `label` = 1 increase significantly (0.6) but at the cost of much lower precision, F1-score and accuracy. Most non-defaulters are now wrongly flagged as defaulters. Adjusting threshold does not quite solve the issue since the model cannot tell defaulters apart confidently. Even at threshold 0.3, only a small group of samples truly correspond to real defaulters.

### 3.3 Random Forest Model with Class-Weight Balancing Instead of SMOTE) 


To address the severe class imbalance in the dataset, we trained a Random Forest with class_weight='balanced'. This approach automatically assigns higher importance to the minority class (defaulters) during training, without generating synthetic samples as in SMOTE.

In [ ]:

rf_model_cw = RandomForestClassifier(
    n_estimators = 200,
    max_depth = 15,
    min_samples_leaf = 5,
    min_samples_split = 10,
    max_features = 'sqrt',
    class_weight = 'balanced',
    random_state = 42,
    n_jobs = -1
)

rf_model_cw.fit(X_train, y_train)


y_pred_cw = rf_model_cw.predict(X_test)
y_pred_proba_cw = rf_model_cw.predict_proba(X_test)[:, 1] 

cv_scores_rf = cross_val_score(rf_model_cw, X_train, y_train, cv = 5, scoring = 'roc_auc')
print(f"Cross-Validation AUC: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std() * 2:.4f})")

precision, recall, threshold = precision_recall_curve(y_test, y_pred_proba_cw)
pr_auc = auc(recall, precision)


print(f"Accuracy: {accuracy_score(y_test, y_pred_cw):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_cw):.4f}")
print(f"PR AUC: {pr_auc:.4f}")
print("\n Classification Report")
print(classification_report(y_test, y_pred_cw))


print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred_cw)
print(cm)

print(f"\nRates:")

total_0 = cm[0,0] + cm[0,1]  
total_1 = cm[1,0] + cm[1,1] 

print(f"\nSpecificity (True Negative Rate): {cm[0,0]/total_0:.4f}")
print(f"False Positive Rate: {cm[0,1]/total_0:.4f}")
print(f"Sensitivity (Recall/True Positive Rate): {cm[1,1]/total_1:.4f}")
print(f"False Negative Rate: {cm[1,0]/total_1:.4f}")



In [ ]:
# Precision recall curve
plt.figure(figsize=(7,5))
plt.plot(recall, precision, color='b', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision–Recall Curve')
plt.legend()
plt.grid(True)
plt.show()

Most of the performance metric values improved quite a bit from the model that uses smote especially recall, false negative rate, pr auc which are most important as having high false negative rate can be quite costly if a defaulter is wrongly predicted as non-defaulter. Even though PR AUC improved slightly, but the model may be doing slightly better than random guessing only because the features might not provide enough information to distinguish between defaulters and non-defaulters. The model still struggles to detect the minority class effectively. Feature engineering will be done under the section 2.4

### 3.4 Random Forest Model  With Feature Engineering

####   3.4.1 Check feature importance

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': X_encoded.columns,
    'importance': rf_model_cw.feature_importances_
}).sort_values('importance', ascending = False)

print(feature_importance.head(10))


Based on the feature importance for the top 10 features, only the top 3 features seem to have predictive power as all other features have importance of less than 0.01. 

#### 3.4.2 Feature Engineering

In [ ]:

import warnings
warnings.filterwarnings('ignore')

def new_features(X):
   
    X_eng = X.copy()
    
    # use ratio
    if 'years_employed' in X.columns and 'age' in X.columns:
        X_eng['career_ratio'] = X['years_employed'] / (X['age'] + 1)
        X_eng['employment_stability'] = X['years_employed'] / (X['age'] - 18 + 1)  # Years since adult
    
    if 'amt_income_log' in X.columns and 'cnt_fam_members' in X.columns:
        X_eng['income_per_member'] = X['amt_income_log'] / (X['cnt_fam_members'] + 1)
        X_eng['log_income_per_member'] = np.log1p(X_eng['income_per_member'])
    
    # interaction terms
    if 'age' in X.columns and 'amt_income_log' in X.columns:
        X_eng['age_income_interaction'] = X['age'] * X['amt_income_log']
        X_eng['age_income_ratio'] = X['age'] / (X['amt_income_log'] + 1)
    
    # nonlinear transf
    if 'age' in X.columns:
        X_eng['age_squared'] = X['age'] ** 2
        X_eng['age_cubed'] = X['age'] ** 3
        X_eng['log_age'] = np.log1p(X['age'])
    
    if 'years_employed' in X.columns:
        X_eng['employment_squared'] = X['years_employed'] ** 2
        X_eng['log_employment'] = np.log1p(X['years_employed'] + 1)
    

    return X_eng

# apply the feature engineering defined above
X_engineered = new_features(X_encoded)

# use one-hot encoding for cat variables
categorical_eng_cols = X_engineered.select_dtypes(include=['object', 'category']).columns
if len(categorical_eng_cols) > 0:
    X_engineered = pd.get_dummies(X_engineered, columns=categorical_eng_cols, drop_first=True)

X_train_eng, X_test_eng, y_train, y_test = train_test_split(
    X_engineered, y, test_size=0.2, random_state=42, stratify=y
)

#  feature selection using correlation with target
correlations = X_train_eng.corrwith(y_train).abs().sort_values(ascending=False)

# select top correlated features
selected_features = correlations[correlations > 0.01].index.tolist()
if len(selected_features) < 20: 
    selected_features = correlations.head(30).index.tolist()

print(f"Selected {len(selected_features)} features for modeling")

X_train_selected = X_train_eng[selected_features]
X_test_selected = X_test_eng[selected_features]



#### 3.4.3 Modelling

In [ ]:
rf_engineered = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_leaf=5,
    min_samples_split=10,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf_engineered.fit(X_train_selected, y_train)


cv_scores = cross_val_score(rf_engineered, X_train_selected, y_train, 
                           cv=5, scoring='roc_auc', n_jobs=-1)
print(f"Cross-Validation AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

y_pred_eng = rf_engineered.predict(X_test_selected)
y_pred_proba_eng = rf_engineered.predict_proba(X_test_selected)[:, 1]



#### 3.4.4 Evaluation of Performance Metrics

In [ ]:
def evaluation(model_name, y_test, y_pred_eng, y_pred_proba_eng, feature_names, model):
    print(f"\n{model_name} Evaluation")


    roc_auc_value = roc_auc_score(y_test, y_pred_proba_eng)
    acc = accuracy_score(y_test, y_pred_eng)
    prec = precision_score(y_test, y_pred_eng)
    rec = recall_score(y_test, y_pred_eng)
    f1 = f1_score(y_test, y_pred_eng)
    pr_precision, pr_recall, _ = precision_recall_curve(y_test, y_pred_proba_eng)
    pr_auc_value = auc(pr_recall, pr_precision)


    print(f"ROC AUC: {roc_auc_value:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"PR AUC:  {pr_auc_value:.4f}") 
    
    # Confusion Matrix
    print(f"\nConfusion Matrix")
    cm = confusion_matrix(y_test, y_pred_eng)
    print(cm)

    print("\n Classification Report")
    print(classification_report(y_test, y_pred_eng))
   
    total_0 = cm[0,0] + cm[0,1]
    total_1 = cm[1,0] + cm[1,1]

    print(f"\nRates:")

    print(f"\nSpecificity (True Negative Rate): {cm[0,0]/total_0:.4f}")
    print(f"False Positive Rate: {cm[0,1]/total_0:.4f}")
    print(f"Sensitivity (Recall/True Positive Rate): {cm[1,1]/total_1:.4f}")
    print(f"False Negative Rate: {cm[1,0]/total_1:.4f}")

    # for plotting in part 2.4.5
    return {
        'roc_auc': roc_auc_value,
        'pr_auc': pr_auc_value,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'confusion_matrix': cm
    }
#1.  evaluate engineered model
results_eng = evaluation(
    "Random Forest with Feature Engineering", 
    y_test, y_pred_eng, y_pred_proba_eng, 
    selected_features, rf_engineered
)


# 2.  train "baseline" model (using class weight (rf_model_cw), no feature engineering)

results_cw = evaluation(
    "Random Forest (No Engineering)", 
    y_test, y_pred_cw, y_pred_proba_cw, 
    X_train.columns, rf_model_cw
)

For the random forest model with feature engineered features, most of the performance metrics improved quite a bit as compared to the baseline model of random forest using class-weighting. 
1. The model is moderately strong. It performs significantly better than random guessing and slightly better than the baseline
2. After feature engineering, the model identifies more true positives with slightly better confidence, however, more needs to be done to reduce the false negative rate which can be very costly error. 

#### 3.4.5 Visualisation

In [ ]:

plt.figure(figsize=(18, 12))

# auc roc curve comparison plot
plt.subplot(2, 3, 1)
fpr_eng, tpr_eng, _ = roc_curve(y_test, y_pred_proba_eng)
fpr_base, tpr_base, _ = roc_curve(y_test, y_pred_proba_cw)

plt.plot(fpr_eng, tpr_eng, label=f'With Feature Engineering (AUC = {results_eng["roc_auc"]:.4f})', linewidth=2)
plt.plot(fpr_base, tpr_base, label=f'Baseline (AUC = {results_cw["roc_auc"]:.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC ROC Curve Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

#  feature importance plot (show new features)
plt.subplot(2, 3, 2)
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': rf_engineered.feature_importances_
}).sort_values('importance', ascending=False).head(15)

sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Top 15 Feature Importances\n(Engineered Model)')
plt.xlabel('Importance Score')

# performance metrics comparison plot
plt.subplot(2, 3, 3)
metrics = ['ROC AUC', 'PR AUC', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
base_values = [results_cw['roc_auc'], results_cw['pr_auc'],results_cw['accuracy'], results_cw['precision'], 
               results_cw['recall'], results_cw['f1']]
eng_values = [results_eng['roc_auc'], results_cw['pr_auc'], results_eng['accuracy'], results_eng['precision'], 
              results_eng['recall'], results_eng['f1']]

x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, base_values, width, label='Baseline', alpha=0.7)
plt.bar(x + width/2, eng_values, width, label='With Engineering', alpha=0.7)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Performance Comparison')
plt.xticks(x, metrics, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# precision recall curve plot
plt.subplot(2, 3, 4)
precision_eng, recall_eng, _ = precision_recall_curve(y_test, y_pred_proba_eng)
precision_base, recall_base, _ = precision_recall_curve(y_test, y_pred_proba_cw)

plt.plot(recall_eng, precision_eng, label='With Feature Engineering', linewidth=2)
plt.plot(recall_base, precision_base, label='Baseline', linewidth=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. XGBOOST Model

#### 3.1 XGBoost Model (version 1)

Rationale of XGBoost with `scale_pos_weight1` & regularisation:
- Class imbalance handling using `scale_pos_weight = (# negatives / # positives)` which penalises mistakes on minority class more (defaulters). This would improve recall on the minority (defaulters).
- Regularisation using `reg_alpha = 0.1` and `reg_lambda = 0.1` to reduce overfitting 

We chose log loss as the evaluation metric, because it trains XGBoost to penalise the model for being confidently wrong (in this case wrongly predicting defaulters as non-defaulters). 

In [ ]:

## XGBoost (with Scale Pos Weight) and with regularisation

xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=15,   
    learning_rate=0.1,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    eval_metric='logloss'
)

# Train
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Cross-validation
cv_scores_xgb = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"Cross-Validation AUC: {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std() * 2:.4f})")

## COMPREHENSIVE EVALUATION FUNCTION
def evaluate_model(model_name, y_true, y_pred, y_pred_proba):
    print(f"\n--- {model_name.upper()} EVALUATION ---")
    
    # Basic metrics
    auc = roc_auc_score(y_true, y_pred_proba)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"AUC: {auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nConfusion Matrix:")
    print(cm)
    print(f"TN: {cm[0,0]}, FP: {cm[0,1]}, FN: {cm[1,0]}, TP: {cm[1,1]}")
    
    # Rates
    total_0 = cm[0,0] + cm[0,1]
    total_1 = cm[1,0] + cm[1,1]
    print(f"Specificity: {cm[0,0]/total_0:.4f}")
    print(f"False Positive Rate: {cm[0,1]/total_0:.4f}")
    print(f"Sensitivity (Recall): {cm[1,1]/total_1:.4f}")
    print(f"False Negative Rate: {cm[1,0]/total_1:.4f}")
    
    return {
        'model': model_name,
        'auc': auc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm
    }

# Evaluate both models
results = []
results.append(evaluate_model("Random Forest With Class Weight", y_test, y_pred_cw, y_pred_proba_cw))
results.append(evaluate_model("XGBoost", y_test, y_pred_xgb, y_pred_proba_xgb))

## COMPARISON SUMMARY

print("\nMODEL COMPARISON SUMMARY")

comparison_df = pd.DataFrame(results)
print(comparison_df[['model', 'auc', 'accuracy', 'precision', 'recall', 'f1']].round(4))



#---------------------------------------


plt.figure(figsize=(15, 5))

# ROC Curve
plt.subplot(1, 3, 1)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_cw)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)

plt.plot(fpr_rf, tpr_rf, label=f'Random Forest with Class Weight (AUC = {roc_auc_score(y_test, y_pred_proba_cw):.4f})')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {roc_auc_score(y_test, y_pred_proba_xgb):.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.grid(True)

# Precision-Recall Curves
plt.subplot(1, 3, 2)
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_pred_proba_cw)
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, y_pred_proba_xgb)

plt.plot(recall_rf, precision_rf, label='Random Forest with Class Weight')
plt.plot(recall_xgb, precision_xgb, label='XGBoost')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()
plt.grid(True)

#  Metrics Comparison
plt.subplot(1, 3, 3)
metrics = ['AUC', 'Accuracy', 'Precision', 'Recall', 'F1-Score']
rf_metrics = [results[0]['auc'], results[0]['accuracy'], results[0]['precision'], 
              results[0]['recall'], results[0]['f1']]
xgb_metrics = [results[1]['auc'], results[1]['accuracy'], results[1]['precision'], 
               results[1]['recall'], results[1]['f1']]

x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, rf_metrics, width, label='Random Forest with Class Weight', alpha=0.7)
plt.bar(x + width/2, xgb_metrics, width, label='XGBoost', alpha=0.7)
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Metrics Comparison')
plt.xticks(x, metrics, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


#### 3.2 XGBoost Model (with GridSearchCV to get optimal hyperparameters)

Then, we used GridSearchCV to tune the model's parameters and identify the best settings using ROC-AUC as the optimisation metric. Finally, we compared both models on the test set using various evaluation metrics: accuracy, ROC-AUC, precision, recall, and confusion matrix.

We chose log loss as the evaluation metric, because it trains XGBoost to penalise the model for being confidently wrong (in this case wrongly predicting defaulters as non-defaulters). 

In [ ]:
# ================================
# 1. BASELINE RANDOM FOREST (no feature engineering and no tuning)
# ================================
rf_model = RandomForestClassifier(random_state=42)  # <- strictly default
rf_model.fit(X_train, y_train)

rf_pred      = rf_model.predict(X_test)
rf_proba     = rf_model.predict_proba(X_test)[:, 1]

print("=== Baseline Random Forest ===")
print("AUC:", roc_auc_score(y_test, y_pred_proba_cw))
print("Accuracy:", accuracy_score(y_test, y_pred_cw))
print(confusion_matrix(y_test, y_pred_cw))


# ================================
# 2. XGBOOST (with feature engineering) from above 
# ================================


# ================================
# 3. TUNED XGBOOST (GridSearchCV)
# ================================

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

# Assumptions (already defined above):
# X_train_selected, X_test_selected  -> FE/selected columns for XGB
# y_train, y_test

# Compute class imbalance weight ON TRAIN ONLY
pos_weight = (y_train == 0).sum() / max(1, (y_train == 1).sum())


# Base estimator (same assumptions as above)
xgb_base_for_grid = XGBClassifier(
    n_estimators=200,          # will be overridden by grid
    max_depth=15,             # will be overridden by grid
    learning_rate=0.1,        # may be overridden by grid
    scale_pos_weight=pos_weight,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1
)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [7, 15],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'reg_alpha': [0.0, 0.1],
    'reg_lambda': [0.0, 0.1]
}

# For imbalanced classification, StratifiedKFold ensures that each fold has a similar positive/negative ratio.

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_xgb = GridSearchCV(
    estimator=xgb_base_for_grid,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=skf,
    n_jobs=-1,
    verbose=2
)

# Run grid search on your feature engineered data
grid_xgb.fit(X_train_selected, y_train)

# Get the best estimator and make predictions on the test set
best_xgb = grid_xgb.best_estimator_
print("Best Parameters:", grid_xgb.best_params_)
print(f"Best Mean CV ROC AUC: {grid_xgb.best_score_:.4f}")

# Predict on TEST (same selected FE features)
y_pred_xgb_tuned   = best_xgb.predict(X_test_selected)
y_proba_xgb_tuned  = best_xgb.predict_proba(X_test_selected)[:, 1]


# Evaluate on TEST
test_roc = roc_auc_score(y_test, y_proba_xgb_tuned)
test_ap  = average_precision_score(y_test, y_proba_xgb_tuned)  # PR-AUC (AP)
print("\n=== Tuned XGBoost (GridSearchCV) ===")
print(f"Test ROC AUC: {test_roc:.4f}")
print(f"Test PR  AUC: {test_ap:.4f}")
print(f"Test Accuracy:  {accuracy_score(y_test, y_pred_xgb_tuned):.4f}")
print(f"Test Precision: {precision_score(y_test, y_pred_xgb_tuned, zero_division=0):.4f}")
print(f"Test Recall:    {recall_score(y_test, y_pred_xgb_tuned, zero_division=0):.4f}")
print(f"Test F1-Score:  {f1_score(y_test, y_pred_xgb_tuned, zero_division=0):.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb_tuned, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb_tuned))

In [ ]:

# ---------------------------------------
# 4. PLOT ROC CURVES FOR ALL MODELS
# ---------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score
)

# Random Forest
y_pred_rf  = rf_pred
y_proba_rf = rf_proba

# Baseline XGBoost (feature-engineered baseline in your flow)
y_pred_xgb_fe  = y_pred_xgb
y_proba_xgb_fe = y_pred_proba_xgb

# Tuned XGBoost (feature-engineered + GridSearchCV)
y_pred_xgb_tuned  = y_pred_xgb_tuned       
y_proba_xgb_tuned = y_proba_xgb_tuned       
# --- ROC Curves ---
fpr_rf,    tpr_rf,    _ = roc_curve(y_test, rf_proba)
fpr_fe,    tpr_fe,    _ = roc_curve(y_test, y_pred_proba_xgb)
fpr_tuned, tpr_tuned, _ = roc_curve(y_test, y_proba_xgb_tuned)

auc_rf    = roc_auc_score(y_test, rf_proba)
auc_fe    = roc_auc_score(y_test, y_pred_proba_xgb)
auc_tuned = roc_auc_score(y_test, y_proba_xgb_tuned)

# --- Precision-Recall Curves ---
prec_rf,    recall_rf,    _ = precision_recall_curve(y_test, rf_proba)
prec_fe,    recall_fe,    _ = precision_recall_curve(y_test, y_pred_proba_xgb)
prec_tuned, recall_tuned, _ = precision_recall_curve(y_test, y_proba_xgb_tuned)

ap_rf    = average_precision_score(y_test, rf_proba)
ap_fe    = average_precision_score(y_test, y_pred_proba_xgb)
ap_tuned = average_precision_score(y_test, y_proba_xgb_tuned)

# --- Metrics for bar plot ---
metrics = ["Accuracy", "Precision", "Recall", "F1-score"]

rf_scores = [
    accuracy_score(y_test, rf_pred),
    precision_score(y_test, rf_pred, zero_division=0),
    recall_score(y_test, rf_pred, zero_division=0),
    f1_score(y_test, rf_pred, zero_division=0)
]
fe_scores = [
    accuracy_score(y_test, y_pred_xgb),
    precision_score(y_test, y_pred_xgb, zero_division=0),
    recall_score(y_test, y_pred_xgb, zero_division=0),
    f1_score(y_test, y_pred_xgb, zero_division=0)
]
tuned_scores = [
    accuracy_score(y_test, y_pred_xgb_tuned),
    precision_score(y_test, y_pred_xgb_tuned, zero_division=0),
    recall_score(y_test, y_pred_xgb_tuned, zero_division=0),
    f1_score(y_test, y_pred_xgb_tuned, zero_division=0)
]

df_metrics = pd.DataFrame({
    "Metric": metrics,
    "RF (no FE)": rf_scores,
    "XGB (FE)": fe_scores,
    "XGB Tuned (FE)": tuned_scores
})

# --- Create subplots ---
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# (1) ROC curve
axes[0].plot(fpr_rf,    tpr_rf,    label=f"RF (AUC = {auc_rf:.3f})")
axes[0].plot(fpr_fe,    tpr_fe,    label=f"XGB (FE) (AUC = {auc_fe:.3f})")
axes[0].plot(fpr_tuned, tpr_tuned, label=f"XGB Tuned (FE) (AUC = {auc_tuned:.3f})")
axes[0].plot([0, 1], [0, 1], linestyle="--", color='gray')  # chance line
axes[0].set_xlabel("False Positive Rate")
axes[0].set_ylabel("True Positive Rate")
axes[0].set_title("ROC Curve")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# (2) Precision–Recall curve
axes[1].plot(recall_rf,    prec_rf,    label=f"RF (AP = {ap_rf:.3f})")
axes[1].plot(recall_fe,    prec_fe,    label=f"XGB (FE) (AP = {ap_fe:.3f})")
axes[1].plot(recall_tuned, prec_tuned, label=f"XGB Tuned (FE) (AP = {ap_tuned:.3f})")
# Baseline = prevalence
axes[1].axhline((y_test == 1).mean(), ls='--', c='gray', lw=1, label="Baseline = prevalence")
axes[1].set_xlabel("Recall")
axes[1].set_ylabel("Precision")
axes[1].set_title("Precision–Recall Curve")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# (3) Bar plot of thresholded metrics
x = np.arange(len(metrics))
width = 0.23
axes[2].bar(x - width,      df_metrics["RF (no FE)"],         width, label="RF (no FE)")
axes[2].bar(x,              df_metrics["XGB (FE)"],           width, label="XGB (FE)")
axes[2].bar(x + width,      df_metrics["XGB Tuned (FE)"],     width, label="XGB Tuned (FE)")
axes[2].set_xticks(x)
axes[2].set_xticklabels(metrics)
axes[2].set_ylabel("Score")
axes[2].set_ylim(0, 1.0)
axes[2].set_title("Metric Comparison @ default threshold")
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle("Model Comparison: RF (no FE) vs XGB (FE) vs XGB Tuned (FE)", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:

rf_scores = [
    accuracy_score(y_test, rf_pred),
    precision_score(y_test, rf_pred, zero_division=0),
    recall_score(y_test, rf_pred, zero_division=0),
    f1_score(y_test, rf_pred, zero_division=0)
]

xgb_fe_scores = [
    accuracy_score(y_test, y_pred_xgb),
    precision_score(y_test, y_pred_xgb, zero_division=0),
    recall_score(y_test, y_pred_xgb, zero_division=0),
    f1_score(y_test, y_pred_xgb, zero_division=0)
]

xgb_tuned_scores = [
    accuracy_score(y_test, y_pred_xgb_tuned),
    precision_score(y_test, y_pred_xgb_tuned, zero_division=0),
    recall_score(y_test, y_pred_xgb_tuned, zero_division=0),
    f1_score(y_test, y_pred_xgb_tuned, zero_division=0)
]

df_metrics = pd.DataFrame({
    "Metric": metrics,
    "RF (no FE)": rf_scores,
    "XGB (FE)": xgb_fe_scores,
    "XGB Tuned (FE)": xgb_tuned_scores
})

print(df_metrics.to_string(index=False))

## Bar Plot Interpretation: Model Evaluation Metrics

The bar plot compares four key evaluation metrics: Accuracy, Precision, Recall, and F1-score, across Random Forest, baseline XGBoost, and tuned XGBoost models on the test set. 
The main task is to identify defaulters (1) versus non-defaulters (0), in a dataset with a strong class imbalance (far fewer defaulters).

1. Accuracy
All three models achieve very high accuracy (around 95.8% to 96%). However, in this imbalanced dataset, where defaulters are much less common than non-defaulters, high accuracy is not a strong indicator of real-world performance. Most of this accuracy comes from correctly predicting the majority class (non-defaulters), while performance on the minority class (defaulters) is what truly matters.

2. Precision, Recall, and F1-score
Precision indicates how often a predicted defaulter is actually a defaulter. The values are fairly low (0.17 to 0.19), reflecting the difficulty in making correct positive predictions due to class imbalance.

Recall measures how many actual defaulters the model successfully identifies. Here, the tuned XGBoost achieves the highest recall (0.455), followed closely by baseline XGBoost (0.447), and then Random Forest (0.366).

F1-score is the harmonic mean of precision and recall, representing the overall balance between the two. The tuned XGBoost has the highest F1-score (0.271), indicating the best trade-off between precision and recall.


Model Comparison:

- Random Forest achieves the lowest precision, recall, and F1-score among the three, but still provides a meaningful baseline.

- Baseline XGBoost improves upon Random Forest, especially in terms of recall and F1-score, suggesting it’s better at finding defaulters and balancing false positives.

- Tuned XGBoost slightly outperforms both Random Forest and baseline XGBoost on all key metrics, demonstrating that hyperparameter tuning has helped the model become even more effective at detecting defaulters.

In the context of credit default prediction, it is crucial to find the right balance between identifying true defaulters and minimizing false alarms. The precision-recall curve is a valuable tool for visualizing this trade-off, as it shows how precision (the proportion of correctly identified defaulters among all predicted defaulters) and recall (the proportion of actual defaulters correctly identified by the model) change as we adjust the classification threshold. In practical terms, if our main concern is to avoid lending to customers who are likely to default, we may prioritize a higher recall, accepting a greater number of false positives to ensure we capture as many defaulters as possible. Conversely, if we want to avoid mistakenly classifying reliable customers as risky, we might set a higher threshold to achieve higher precision, even if it means some defaulters go undetected. By analyzing the precision-recall curve, we can select a threshold that aligns with our institution’s risk appetite and business objectives, ensuring our credit decision process is both responsible and aligned with our strategic goals.


## Learning curve 

Plot that shows the relationship between the size of training dataset and the model's performance on both training and validation/test set. It tells us how the model’s performance improves (or stagnates) as it sees more data.

1. XGBoost can be prone to overfitting if not tuned well, thus, we displayed the learning curve to see if the current hyperparameter settings is fitting the data well.
- High training score but low validation score suggests model is overfitting.
- Low training and validation scores suggest model is underfitting.


2. See if adding more data would help:
- If validation score is still improving with more data, more samples might help.



In [ ]:

from sklearn.model_selection import learning_curve, StratifiedKFold

# --- Unfitted model instances (match your training runs) ---
rf_for_curve = RandomForestClassifier(random_state=42)  # baseline RF (no FE)

xgb_base_for_curve = XGBClassifier(
    n_estimators=200, max_depth=15, learning_rate=0.1,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
    subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=0.1,
    random_state=42, eval_metric='logloss', n_jobs=-1
)

# clone tuned params but keep it UNFITTED for learning_curve
tuned_xgb_for_curve = XGBClassifier(**best_xgb.get_params())

# --- Stratified CV for imbalance ---
skf5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# --- Learning curve helper (accepts X, y explicitly) ---
def get_curve(estimator, X, y):
    sizes, train_scores, val_scores = learning_curve(
        estimator=estimator, X=X, y=y,
        train_sizes=np.linspace(0.1, 1.0, 5),
        cv=skf5, scoring='roc_auc', n_jobs=-1,
        shuffle=True, random_state=42
    )
    return sizes, val_scores.mean(axis=1), val_scores.std(axis=1)

# RF on RAW features; XGBs on FE/selected features
sizes_rf,    val_rf,    std_rf    = get_curve(rf_for_curve,        X_train,           y_train)
sizes_base,  val_base,  std_base  = get_curve(xgb_base_for_curve,  X_train_selected,  y_train)
sizes_tuned, val_tuned, std_tuned = get_curve(tuned_xgb_for_curve, X_train_selected,  y_train)

# --- Plot (unchanged, just using val_* / std_*) ---
plt.figure(figsize=(8, 6))
plt.fill_between(sizes_rf,    val_rf - std_rf,    val_rf + std_rf,    alpha=0.1)
plt.fill_between(sizes_base,  val_base - std_base,  val_base + std_base,  alpha=0.1)
plt.fill_between(sizes_tuned, val_tuned - std_tuned, val_tuned + std_tuned, alpha=0.1)

plt.plot(sizes_rf,   val_rf,   'o-', label="RF (no FE)",      zorder=3)
plt.plot(sizes_base, val_base, 's--', label="XGB (FE)",       zorder=2)
plt.plot(sizes_tuned,val_tuned,'d-.', label="XGB Tuned (FE)", zorder=4)

plt.xlabel('Number of Training Samples')
plt.ylabel('Validation ROC AUC (Stratified 5-fold)')
plt.title('Learning Curves: RF (no FE) vs XGB (FE) vs XGB Tuned (FE)')
plt.legend()
plt.tight_layout()
plt.show()




X-axis: Number of training samples used (from small to large subsets of your data).

Y-axis: Validation ROC AUC (how well the model ranks positive vs negative cases on held-out folds).

Curves: Each line shows how each model’s performance improves as more data is used for training.

Shaded area: ±1 standard deviation across CV folds, showing variability.


Interpretation
1. Performance Improves with More Data
All models show increasing validation ROC AUC as the number of training samples increases. This is expected—more data generally leads to better generalization.

2. Random Forest Outperforms XGBoost at All Sample Sizes

3. Baseline and Tuned XGBoost are Similar, but Tuned Overtakes at Larger Size
As the sample size increases, the tuned XGBoost pulls ahead slightly, showing the benefit of hyperparameter tuning

4. Variance Decreases with More Data
The shaded regions (standard deviation) are widest at the smallest sample sizes and narrow as more data is used. This indicates more stable, reliable performance estimates at higher sample sizes.

5. All Models Have Not Plateaued
The ROC AUC still increases as you approach the largest sample size. This suggests that even more data could yield further gains, none of the models have fully saturated their learning capacity.